In [2]:
import pandas as pd
import requests

In [32]:
count = 1
ID_list = []
while(count >= 1):
  url = f"https://www.99acres.com/api-aggregator/srp/search?property_type=1&area_unit=1&platform=DESKTOP&moduleName=GRAILS_SRP&workflow=GRAILS_SRP&page_size=30&page={count}&city=25&preference=S&res_com=R&seoUrlType=DEFAULT"
  response = requests.get(url)
  response = response.json()
  if response['status']['code'] != 1:
      break
  else:
    ids = response['properties']
    c = 1
    for id in ids:
      if c != 31:
        ID_list.append(id['PROP_ID'])
        c += 1
  count += 1

print(len(ID_list))

16800


In [ ]:
output = pd.DataFrame()

for id in ID_list:
  url = f"https://www.99acres.com/api-aggregator/psuedoData?prop_id={id}&stage=load"
  response = requests.get(url)
  response = response.json()

  prop_data = response['propertyDetails']['prop_data']

  lattitude = prop_data['Latitude']
  longitude = prop_data['Longitude']
  post_date = prop_data['Modify_Date']
  availability_date = prop_data['Availability_Info']['Availability_Date']
  owner_name = prop_data['company_label']
  property_name = prop_data['Prop_Name']
  id = prop_data['Building_Id']
  locality_name = prop_data ['Locality_Name']
  base_price = prop_data['Price_Per_Unit_Area']

  try:
      for floor in response['floorPlan']:
        area_sqft = floor['SALEABLE_AREA_SQFT']
        price = floor['NEW_BOOKING_PRICE']['min']
        details = floor['unitDetailSectionParts']
        bedrooms = details['bedrooms']
        kitchens = details['kitchens']
        bathroom = details['bathroom']
        balcony = details['balcony']
        living = details['living']
        data = {'id': id, 
                'lattitude': lattitude, 
                'longitude': longitude, 
                'post_date': post_date, 
                'availability_date': availability_date, 
                'owner_name': owner_name, 
                'property_name': property_name, 
                'locality_name': locality_name, 
                'base_price': base_price, 
                'area_sqft': area_sqft, 
                'price': price, 
                'bedrooms': bedrooms, 
                'kitchens': kitchens, 
                'bathroom': bathroom, 
                'balcony': balcony, 
                'living': living}
        output = output.append(data, ignore_index=True)
  except:
    pass

In [90]:
output.head()

,area_sqft,availability_date,balcony,base_price,bathroom,bedrooms,id,kitchens,lattitude,living,locality_name,longitude,owner_name,post_date,price,property_name,url
0,1361.0,"Oct 01, 2023",None,7391,None,3.0,768153,0.0,22.499612,None,E M Bypass,88.408207,Natural Group & Shivom Realty,"Nov 02, 2020",10000000.0,Shivom Utopia,https://www.99acres.com/3-bhk-bedroom-apartmen...
1,1373.0,"Oct 01, 2023",None,7391,None,3.0,768153,0.0,22.499612,None,E M Bypass,88.408207,Natural Group & Shivom Realty,"Nov 02, 2020",10100000.0,Shivom Utopia,https://www.99acres.com/3-bhk-bedroom-apartmen...
2,1531.0,"Oct 01, 2023",None,7391,None,3.0,768153,0.0,22.499612,None,E M Bypass,88.408207,Natural Group & Shivom Realty,"Nov 02, 2020",11200000.0,Shivom Utopia,https://www.99acres.com/3-bhk-bedroom-apartmen...
3,1532.0,"Oct 01, 2023",None,7391,None,3.0,768153,0.0,22.499612,None,E M Bypass,88.408207,Natural Group & Shivom Realty,"Nov 02, 2020",11200000.0,Shivom Utopia,https://www.99acres.com/3-bhk-bedroom-apartmen...
4,1704.0,"Oct 01, 2023",None,7391,None,3.0,768153,0.0,22.499612,None,E M Bypass,88.408207,Natural Group & Shivom Realty,"Nov 02, 2020",13000000.0,Shivom Utopia,https://www.99acres.com/3-bhk-bedroom-apartmen...


In [91]:
output.shape

(693, 17)

In [92]:
output.to_csv(r'Datasets\raw_dataset.csv')